In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
# from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50 , EfficientNetB0 , MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model


# Préparer les données avec l'augmentation
On utilise ImageDataGenerator pour prétraiter et augmenter les images

In [32]:
# Répertoires des données
train_dir = '/content/drive/MyDrive/dataset_echo_sopk/train'
val_dir  = '/content/drive/MyDrive/dataset_echo_sopk/test'

img_size = (224, 224)
batch_size = 16  #32


# Générateurs de données avec augmentation pour l'entraînement
train_datagen = ImageDataGenerator(
    # rescale=1. / 255,
    # rotation_range=45,
    # width_shift_range=0.3,
    # height_shift_range=0.3,
    # shear_range=0.5,
    # zoom_range=0.5,
    # horizontal_flip=True,
    # vertical_flip=True,
    # fill_mode='nearest'
     rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    horizontal_flip=True,
    validation_split=0.2
)
# Générateur de validation (pas d'augmentation ici)
val_datagen = ImageDataGenerator(rescale=1./255)

# Charger les images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary'
)

print(train_generator.class_indices)


Found 2928 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
{'infected': 0, 'notinfected': 1}


In [33]:

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Définition des callbacks pour éviter l'overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)



# CNN

In [34]:
#  Construire le modèle CNN

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # pour la classification binaire
])


In [37]:
# Compiler et Entraîner le modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    validation_data = val_generator ,
    epochs=10,  # Modifier selon les besoins
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 51s 265ms/step - accuracy: 0.9944 - loss: 0.0335 - val_accuracy: 1.0000 - val_loss: 3.1802e-04 - learning_rate: 0.0010
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 79s 255ms/step - accuracy: 0.9946 - loss: 0.0139 - val_accuracy: 1.0000 - val_loss: 7.3682e-05 - learning_rate: 0.0010
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 46s 250ms/step - accuracy: 0.9982 - loss: 0.0052 - val_accuracy: 1.0000 - val_loss: 2.1307e-05 - learning_rate: 0.0010
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 47s 257ms/step - accuracy: 1.0000 - loss: 2.0623e-04 - val_accuracy: 1.0000 - val_loss: 5.2982e-06 - learning_rate: 0.0010
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 46s 251ms/step - accuracy: 0.9999 - loss: 5.5425e-04 - val_accuracy: 1.0000 - val_loss: 1.2038e-05 - learning_rate: 2.0000e-04
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 48s 262ms/step - accuracy: 1.0000 - loss: 8.8289e-04 - val_accuracy: 1.0000 - val_loss: 1.2130e-06 - learning_rate: 2.0000e-04
Epoch 7/10
183/183 ━━━━━━━

In [38]:
# Évaluer le modèle
test_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_echo_sopk/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

loss, acc = model.evaluate(test_generator)
print(f'Test Accuracy: {acc * 100:.2f}%')


Found 300 images belonging to 2 classes.
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 246ms/step - accuracy: 1.0000 - loss: 3.0828e-06
Test Accuracy: 100.00%


In [39]:
import cv2
import numpy as np

# Chemin de l'image à tester
img_path = "/content/drive/MyDrive/dataset_echo_sopk/test_final/notinfected1.jpg"

# Charger et préparer l'image
image = cv2.imread(img_path)                         # Lire l'image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)       # Convertir BGR -> RGB
image = cv2.resize(image, (224, 224))                # Redimensionner à la taille du modèle
image = image / 255.0                                # Normaliser les pixels entre 0 et 1
image = np.expand_dims(image, axis=0)                # Ajouter la dimension batch (1, 224, 224, 3)

# Prédiction
prediction = model.predict(image)[0][0]

# Résultat
if prediction < 0.5:
    print(f"L'ovaire est infecté (SOPK détecté) avec {(1 - prediction) * 100:.2f}% de certitude")
else:
    print(f"L'ovaire est sain avec {prediction * 100:.2f}% de certitude")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
L'ovaire est sain avec 100.00% de certitude


# ResNet50

In [40]:
# Charger ResNet50 + ajouter des couches finales
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Ajouter des couches finales
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # binaire : malade / non

model = Model(inputs=base_model.input, outputs=output)


In [41]:
# Compiler et entraîner
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    # callbacks=[early_stop]
     callbacks=[early_stopping, reduce_lr]
)

Epoch 1/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 69s 327ms/step - accuracy: 0.5352 - loss: 0.7399 - val_accuracy: 0.8500 - val_loss: 0.5683 - learning_rate: 1.0000e-04
Epoch 2/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 49s 268ms/step - accuracy: 0.7087 - loss: 0.6029 - val_accuracy: 0.8733 - val_loss: 0.5590 - learning_rate: 1.0000e-04
Epoch 3/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 48s 264ms/step - accuracy: 0.7980 - loss: 0.5207 - val_accuracy: 0.9100 - val_loss: 0.4794 - learning_rate: 1.0000e-04
Epoch 4/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 50s 271ms/step - accuracy: 0.8440 - loss: 0.4551 - val_accuracy: 0.8567 - val_loss: 0.4769 - learning_rate: 1.0000e-04
Epoch 5/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 51s 278ms/step - accuracy: 0.8851 - loss: 0.3828 - val_accuracy: 0.9233 - val_loss: 0.3945 - learning_rate: 1.0000e-04
Epoch 6/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 49s 267ms/step - accuracy: 0.8901 - loss: 0.3457 - val_accuracy: 0.9533 - val_loss: 0.2852 - learning_rate: 1.0000e-04
Epoch 7/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 48s 26

In [16]:
# Évaluer le modèle
test_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_echo_sopk/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

loss, acc = model.evaluate(test_generator)
print(f'Test Accuracy: {acc * 100:.2f}%')

Found 300 images belonging to 2 classes.
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 245ms/step - accuracy: 0.9754 - loss: 0.1292
Test Accuracy: 97.67%


In [12]:
# model.save("model_echo_sopk.keras")
# model.save("model_echo_sopk.h5")

In [ ]:
# model.save("model_echographie_sopk.keras")

In [18]:
import cv2
import numpy as np

# Chemin de l'image à tester
img_path = "/content/drive/MyDrive/dataset_echo_sopk/test_final/notinfected1.jpg"

# Charger et préparer l'image
image = cv2.imread(img_path)                         # Lire l'image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)       # Convertir BGR -> RGB
image = cv2.resize(image, (224, 224))                # Redimensionner à la taille du modèle
image = image / 255.0                                # Normaliser les pixels entre 0 et 1
image = np.expand_dims(image, axis=0)                # Ajouter la dimension batch (1, 224, 224, 3)

# Prédiction
prediction = model.predict(image)[0][0]

# Résultat
if prediction < 0.5:
    print(f"L'ovaire est infecté (SOPK détecté) avec {(1 - prediction) * 100:.2f}% de certitude")
else:
    print(f"L'ovaire est sain avec {prediction * 100:.2f}% de certitude")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
L'ovaire est sain avec 93.52% de certitude


# **EfficientNetB0**

In [20]:
# Charger EfficientNetB0 + ajouter des couches finales
# Charger EfficientNetB0 sans la tête finale
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler les couches du backbone
for layer in base_model.layers:
    layer.trainable = False

# Ajouter nos propres couches
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # Binaire

model = Model(inputs=base_model.input, outputs=output)


In [21]:
# Compiler et entraîner
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    # callbacks=[early_stop]
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 81s 323ms/step - accuracy: 0.5212 - loss: 0.6969 - val_accuracy: 0.6667 - val_loss: 0.6480 - learning_rate: 1.0000e-04
Epoch 2/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 47s 254ms/step - accuracy: 0.5127 - loss: 0.6954 - val_accuracy: 0.6667 - val_loss: 0.6517 - learning_rate: 1.0000e-04
Epoch 3/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 48s 261ms/step - accuracy: 0.5180 - loss: 0.6924 - val_accuracy: 0.6667 - val_loss: 0.6571 - learning_rate: 1.0000e-04
Epoch 4/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 50s 272ms/step - accuracy: 0.5028 - loss: 0.6977 - val_accuracy: 0.6667 - val_loss: 0.6654 - learning_rate: 1.0000e-04


In [22]:
# Évaluer le modèle
test_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_echo_sopk/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

loss, acc = model.evaluate(test_generator)
print(f'Test Accuracy: {acc * 100:.2f}%')

Found 300 images belonging to 2 classes.
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 294ms/step - accuracy: 0.6786 - loss: 0.6437
Test Accuracy: 66.67%


In [23]:
import cv2
import numpy as np

# Chemin de l'image à tester
img_path = "/content/drive/MyDrive/dataset_echo_sopk/test_final/notinfected1.jpg"

# Charger et préparer l'image
image = cv2.imread(img_path)                         # Lire l'image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)       # Convertir BGR -> RGB
image = cv2.resize(image, (224, 224))                # Redimensionner à la taille du modèle
image = image / 255.0                                # Normaliser les pixels entre 0 et 1
image = np.expand_dims(image, axis=0)                # Ajouter la dimension batch (1, 224, 224, 3)

# Prédiction
prediction = model.predict(image)[0][0]

# Résultat
if prediction < 0.5:
    print(f"L'ovaire est infecté (SOPK détecté) avec {(1 - prediction) * 100:.2f}% de certitude")
else:
    print(f"L'ovaire est sain avec {prediction * 100:.2f}% de certitude")


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
L'ovaire est sain avec 59.06% de certitude


# MobileNetV2

In [24]:
# Charger MobileNetV2 + ajouter des couches personnalisées
# Charger MobileNetV2 sans la tête (classification)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Ajouter les couches de classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [25]:
#  Compiler et entraîner
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    # callbacks=[early_stop]
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 63s 293ms/step - accuracy: 0.8449 - loss: 0.3438 - val_accuracy: 1.0000 - val_loss: 0.0255 - learning_rate: 1.0000e-04
Epoch 2/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 45s 248ms/step - accuracy: 0.9989 - loss: 0.0229 - val_accuracy: 1.0000 - val_loss: 0.0078 - learning_rate: 1.0000e-04
Epoch 3/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 47s 255ms/step - accuracy: 0.9996 - loss: 0.0084 - val_accuracy: 1.0000 - val_loss: 0.0036 - learning_rate: 1.0000e-04
Epoch 4/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 46s 249ms/step - accuracy: 1.0000 - loss: 0.0054 - val_accuracy: 1.0000 - val_loss: 0.0022 - learning_rate: 1.0000e-04
Epoch 5/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 47s 255ms/step - accuracy: 1.0000 - loss: 0.0028 - val_accuracy: 1.0000 - val_loss: 0.0015 - learning_rate: 1.0000e-04
Epoch 6/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 46s 253ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 1.0000 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 7/15
183/183 ━━━━━━━━━━━━━━━━━━━━ 49s 26

In [42]:
# Évaluer le modèle
test_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_echo_sopk/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

loss, acc = model.evaluate(test_generator)
print(f'Test Accuracy: {acc * 100:.2f}%')

Found 300 images belonging to 2 classes.
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 283ms/step - accuracy: 0.9647 - loss: 0.1437
Test Accuracy: 96.00%


In [43]:
import cv2
import numpy as np

# Chemin de l'image à tester
img_path = "/content/drive/MyDrive/dataset_echo_sopk/test_final/notinfected1.jpg"

# Charger et préparer l'image
image = cv2.imread(img_path)                         # Lire l'image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)       # Convertir BGR -> RGB
image = cv2.resize(image, (224, 224))                # Redimensionner à la taille du modèle
image = image / 255.0                                # Normaliser les pixels entre 0 et 1
image = np.expand_dims(image, axis=0)                # Ajouter la dimension batch (1, 224, 224, 3)

# Prédiction
prediction = model.predict(image)[0][0]

# Résultat
if prediction < 0.5:
    print(f"L'ovaire est infecté (SOPK détecté) avec {(1 - prediction) * 100:.2f}% de certitude")
else:
    print(f"L'ovaire est sain avec {prediction * 100:.2f}% de certitude")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
L'ovaire est sain avec 94.35% de certitude
